# Indexación de vectores

## Librerías

In [ ]:
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import SQLRecordManager, index
from langchain.schema import Document
from langchain.vectorstores import Chroma

from src.langchain_docs_loader import load_langchain_docs_splitted

load_dotenv()  # It should output True

## Carga de datos

In [ ]:
docs = load_langchain_docs_splitted()
f"Loaded {len(docs)} documents"

## Inicialización de componentes de un índice

Para crear un `índice` es necesario inicializar:

- `Vectorstore`: para almacenar los vectores de los documentos.
- `Record Manager`: para almacenar qué vectores han sidoe indexados y cuándo.

In [ ]:
collection_name = "langchain_docs_index"
embeddings = OpenAIEmbeddings()
vector_store = Chroma(
    collection_name=collection_name,
    embedding_function=embeddings,
)

> Es una buena practica inicializar el `namespace` de nuestro `Record Manager` con el nombre de nuestra `Vectorstore` y el nombre del `Collection` que estamos indexando.

## Indexación

### Función de utilidad para limpiar nuestro índice

In [ ]:
def clear_index():
    """Hacky helper method to clear content. See the `full` mode section to to understand why it works."""
    index(
        [],
        record_manager=record_manager,
        vector_store=vector_store,
        cleanup="full",
        source_id_key="source",
    )

### Indexación con limpieza de tipo `None`

Esta implementación es la opción por defecto. La especificación **no** remueve los documentos previamente indexados. Sin embargo, sí se encarga de de remover los documentos duplicados **antes** de indexarlos.

Si dentro de un tiempo volvemos a ejecutar nuestro código de carga de datos y los documentos ya existen en el índice, no se volverán a indexar.

In [ ]:
index(
    docs_source=docs,
    record_manager=record_manager,
    vector_store=vector_store,
    source_id_key="source",
    cleanup=None,
)

In [ ]:
clear_index()

### Indexación con limpieza de tipo `incremental`

Al igual que la limpieza de tipo `None`, la limpieza `incremental` maneja los documentos duplicados **antes** de indexarlos. Sin embargo, si alguno de los vectores de un **source / recurso** es diferente al que ya existe en el índice, se reemplazará el vector existente por el nuevo.

Si llamamos a la función nuevamente, pero sin ningún documento, entonces nada se eliminará del índice.

In [ ]:
index(
    docs_source=[],
    record_manager=record_manager,
    vector_store=vector_store,
    source_id_key="source",
    cleanup="incremental",
)

Si agregamos un nuevo documento, entonces se indexará.

In [ ]:
index(
    docs_source=,
    record_manager=record_manager,
    vector_store=vector_store,
    source_id_key="source",
    cleanup="incremental",
)

Y si modificamos un documento existente, entonces se reemplazará el vector existente por el nuevo.

In [ ]:
index(
    docs_source=,
    record_manager=record_manager,
    vector_store=vector_store,
    source_id_key="source",
    cleanup="incremental",
)

In [ ]:
clear_index()

### Indexación con limpieza de tipo `full`

Cualquier documento que no sea parte de la carga actual será eliminado del índice. Esto es útil cuando se quiere mantener el índice actualizado con los documentos que se encuentran en el origen de datos. Los documentos que no han sido modificados no serán indexados nuevamente.

In [ ]:
index(
    docs_source=docs,
    record_manager=record_manager,
    vector_store=vector_store,
    source_id_key="source",
    cleanup="full",
)

Indexemos nuevamente los documentos, pero sólo con una pequeña proporción de los mismos.

In [ ]:
index(
    docs_source=,
    record_manager=record_manager,
    vector_store=vector_store,
    source_id_key="source",
    cleanup="full",
)

La función `clear_index` es un caso de uso para la limpieza de tipo `full`

In [ ]:
clear_index()

### Indexación a partir de un `BaseLoader` de `Langchain`

Langchain estable el concepto de `BaseLoader` como clases que se encargan de cargar datos de diferentes fuentes de datos o con un procesamiento específico. Estos pueden ser extendidos para crear `Loaders` personalizados. Y, a su vez, ser utilizados para indexar documentos dentro de nuestro `pipeline` de ingesta.

In [ ]:
from langchain.document_loaders.base import BaseLoader


class MyDocumentLoader(BaseLoader):
    """Here should be the logic to load the documents from the source.

    The `load` method should return a list of `Document` objects.

    In this example, we will just return the `docs` variable defined above.
    """

    def load(self) -> list[Document]:
        return docs

In [ ]:
index(
    docs_source=MyDocumentLoader(),
    record_manager=record_manager,
    vector_store=vector_store,
    source_id_key="source",
    cleanup="full",
)

In [ ]:
clear_index()